#### Search Engine with Agents and Tools

In [1]:
## Tools creation -> Arxiv

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [6]:
## Inbuilt tool of Wikipedia

wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [7]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [8]:
tools = [wiki, arxiv]

In [9]:
## Build own custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector_store = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fd0ecfbf820>, search_kwargs={})

In [10]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [11]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/conda/envs/myenv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 StructuredTool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x7fd0ed109b40>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x7fd0ed109a20>)]

In [ ]:
## Run this tools with Agents and LLMs
from dotenv import load_dotenv

from langchain_groq import ChatGroq

load_dotenv()

groq_api_key = "Your_API_KEY"

model = ChatGroq(model="qwen/qwen3-32b", groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7fd0ecedd060>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fd0ecedcf70>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
## Create prompt template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_classic import hub
from langchain_classic.agents import initialize_agent, AgentType

prompt = hub.pull("hwchase17/react")


In [59]:
## Use Agents
from langchain_classic.agents import create_react_agent

agent = create_react_agent(llm=model, prompt=prompt, tools=tools)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: 'str', callbacks: 'Callbacks' = None) -> 'str | tuple[str, list[Document]]' - Search any information about Langsmith", 'tool_names': 'wikipedia, arxiv, langsmith-search'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash'

In [60]:
## Agent executor

from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith-search(query: 'str', callbacks: 'Callbacks' = None) -> 'str | tuple[str, list[Document]]' - Search any information about Langsmith", 'tool_names': 'wikipedia, arxiv, langsmith-search'}, metadata={'lc_hub_owner': 

In [63]:
agent_executor.invoke({
    "input": "what is the paper 1706.03762 about?"
})



> Entering new AgentExecutor chain...
<think>
Okay, let's see. The user is asking about the paper with ID 1706.03762. I need to figure out what this paper is about. First, I remember that arXiv assigns IDs in the format year.month, so 1706 would be June 2017. The full ID is probably 1706.03762.

Since the user wants information about this specific paper, the best tool to use here is arxiv. The arxiv wrapper can search for papers by their ID. So I should use the arxiv tool with the query '1706.03762' to get details.

Let me execute that. Using arxiv with the input '1706.03762' should retrieve the title, authors, and abstract. Once I have that information, I can summarize it for the user.

Wait, I should check if the arxiv tool can handle the full ID. Sometimes, people might input just the first part, but the tool might need the full arXiv ID. However, in this case, the user provided the full ID, so it should work. Let me proceed.
</think>

Action: arxiv
Action Input: 1706.03762Publish

{'input': 'what is the paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" (arXiv:1706.03762) presents the Transformer, a neural network architecture that uses self-attention to handle sequence transduction tasks. It replaced traditional recurrent/convolutional models, enabling parallel processing and achieving state-of-the-art results in NLP tasks like machine translation. The work by Vaswani et al. (2017) laid the foundation for modern language models.'}